In [ ]:
# PyCaretのインストール
%%capture
!pip install pycaret

# Notebookの中で複数の結果を表示できるようにする
from IPython import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [1]:
# 必要なライブラリのインポート
from pycaret.classification import *
from google.colab import drive
import pandas as pd
import pickle

pd.set_option('display.max_columns', 150)
# データの読み込み
path = '/content/drive/MyDrive/Colaboratory/JAINBP_2022/'
file_name = 'df_for_pycaret.pickle'
with open(path + file_name, 'rb') as f:
    df = pickle.load(f)

# データフレームの確認
df.head()

In [2]:
# 解析データの作成
## 不要な列の削除
## タイムポイント1～3のデータから，
## タイムポイント4のESA・鉄の投与量の変化を予測をしたい．
## ↓
## タイムポイント4の検査データを削除
col_names_base = ['WBC', 'RBC', 'Hgb', 'Ht', 'MCV', 'MCH', 'MCHC', 'RDW', 
              'Alb', 'CRP',
              'TSAT', 'Fer', 'eqESA_dose', 'Fe_dose']
col_names_remove = [x + '_4' for x in col_names_base]
df_rem_col = df
for col_name_remove in col_names_remove:
    df_rem_col = df_rem_col.drop(col_name_remove, axis = 1)
print('タイムポイント4の検査データ削除後の列名一覧')
df_rem_col.columns

# 鉄の予測→タイムポイント4のESA投与量変化を除去
df_fe = df_rem_col.drop('ESA_change_4', axis = 1)
print('\n'*2 + 'タイムポイント4のESA投与量変化データ削除後の列名一覧')
df_fe.columns

# ESAの予測→タイムポイント4の鉄投与量変化を除去
df_esa = df_rem_col.drop('Fe_change_4', axis = 1)
print('\n'*2 + 'タイムポイント4の鉄投与量変化データ削除後の列名一覧')
df_esa.columns

In [3]:
# データの前処理
exp1 = setup(df_fe, session_id = 0, target = 'Fe_change_4')

In [4]:
# 各機械学習手法のperformance比較
compare_models()

In [5]:
# 個別のモデルを作成する
model_created = create_model('et')

In [6]:
# モデルの評価を行う（様々な指標：confusion matrix，feature importance plot...)
evaluate_model(model_created)

In [7]:
# モデルのチューニング（ハイパーパラメータの調整）
tuned_model = tune_model(model_created)

In [8]:
# チューニングされたモデルの評価
evaluate_model(tuned_model)

In [9]:
# 作成されたモデルで値を予測→鉄減量(Labe == -1)と予測されたデータを10件表示
result = predict_model(model_created, data = df_fe)
result[result['Label'] == -1][:10]

In [10]:
# 作成されたモデルで値を予測→鉄増量(Labe == 1)と予測されたデータを10件表示
result = predict_model(tuned_model, data = df_fe)
result[result['Label'] == 1][:10]